# SPFRx Start
##### Last Updated: 20/03/24

This demo will show 

### Pre-requisites

This notebook has the following assumptions and pre-requisites:
* A deployment that includes TMC, CSP.LMC, and CBF is running. For example, one in a namespace `NS` that was launched from the [ska-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline.
* Taranta should be enabled in the deployment and the CSP.LMC Monitoring Dashboard can be used to monitor the relevant CSP.LMC and CBF devices.
* A virtual environment is being used. This notebook was made with Python 3.10 in mind.
* All requirements are installed via [poetry]().
* The talon boards `TARGET_TALONS` are powered off.

## Execution Steps

### Initial Parameter Set Up

The parameters below (except for `TANGO_HOST`) may need to be updated to match your deployment, environment, and paths.

In [20]:
import os

print("Setting up parameters")

TARGET_TALONS = "7,8"
NS = "ci-ska-mid-psi-1227247907-jaredmda"
# BITE_MAC_ADDRESS = "08:c0:eb:9d:47:78"

# TANGO HOST set up
CLUSTER_DOMAIN = "cluster.local"
TANGO_DB = "databaseds-tango-base"
TANGO_HOST = f"{TANGO_DB}.{NS}.svc.{CLUSTER_DOMAIN}:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

# Config files set up
HW_CONFIG_FILE = "../configs/common/hw_config/hw_config_swap_psi.yaml"
# DISH_CONFIG_FILE = "load_dish_config.json"
INIT_SYS_PARAM_FILE = "../configs/common/sys_params/initial_system_param.json"
# SLIM_CONFIG_FILE = "fs_slim_4vcc_1fsp.yaml"
# ASSIGN_RESOURCES_FILE = "assign_resources_tmc.json"
CONFIGURE_SCAN_FILE = "configure_scan.json"
SPFRX_RELEASE = "0.2.1"
RAW_USER_ACCOUNT = ""
RAW_USER_PASS = ""
SPFRX_ADDRESS = "192.168.8.200"
SPFRX_BIN = "/usr/local/bin"
SPFRX_TANGO_INSTANCE = "spfrx-20"
SPFRX_DEFAULT_LOGGING_LEVEL = 4
SPFRX_SCRIPTS_DIR = os.path.abspath("../configs/spfrx/")
SPFRX_MOUNT_DIR = os.path.abspath("../configs/spfrx/mnt/spfrx-config")
# SCAN_FILE = "scan.json"
# RELEASE_RESOURCES_FILE = "release_resources.json"

Setting up parameters


### Initial Set Up Using Engineering Console Deployer

Engineering Console Deployer is used to generate the talondx-config file, download the necessary artifacts from CAR, and configure the database.

In [2]:
print("Copying the HW config file to the CBF Controller pod")
!kubectl cp {HW_CONFIG_FILE} {NS}/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

print(f"\nGenerating the talondx-config file for boards={TARGET_TALONS}")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --generate-talondx-config --boards={TARGET_TALONS}

print("\nDownloading artifacts")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --download-artifacts

print("\nConfiguring the TANGO database")
!kubectl exec -ti -n {NS} ec-deployer -- python3 midcbf_deployer.py --config-db

print("\nDONE")

Copying the HW config file to the CBF Controller pod

Generating the talondx-config file for boards=7,8
[talondx.py: line 487]INFO: User: root
[talondx.py: line 542]INFO: Generate talondx-config.json file

[talondx.py: line 487]INFO: User: root
[talondx.py: line 536]INFO: Download Artifacts
[talondx.py: line 330]INFO: Conan version: 1.62.0
[talondx.py: line 332]INFO: Conan local cache: Existing package recipes:

NE10/1.2.1@nrc/stable
dishidutils/1.0.0@nrc/stable
ds-histogram/1.2.1@nrc/stable
ds-resampler-delay-tracker/1.5.3@nrc/stable
dscircuitswitch/2.1.0@nrc/stable
dscorrelator/2.1.0@nrc/stable
dsdct/1.3.0@nrc/stable
dsfinechannelizer/1.3.0@nrc/stable
dsgaussiannoisegen/2.0.0@nrc/stable
dshostlutstage1/1.4.0@nrc/stable
dshostlutstage2/1.7.0@nrc/stable
dshpsmaster/1.0.6@nrc/stable
dslstvbandpassfilter/1.4.0@nrc/stable
dslstvgen/1.4.0@nrc/stable
dslstvplayback/1.4.0@nrc/stable
dslstvtonegenerator/1.4.0@nrc/stable
dspacketstreamrepair/1.3.0@nrc/stable
dspolarizationcoupler/1.4.0@nrc/sta

### Set Up and Configure Device Proxies 

In [3]:
from PyTango import DeviceProxy

print("Setting up the device proxies")

## TMC proxies
# tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
# tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
# tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")

## CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray= DeviceProxy("mid-csp/subarray/01")

## CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0
cbf_controller.adminMode = 0
cbf_subarray.adminMode = 0

print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.simulationMode = 0
cbf_controller.simulationMode = 0
csp_control.commandTimeout = 99

print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CSP Control's CBF Simulation Mode: {bool(csp_control.simulationMode)}")
print(f"  CBF Simulation Mode: {bool(cbf_controller.simulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

Setting up the device proxies


omniORB: (0) 2024-04-02 17:05:17.482775: ORB_init failed: Bad parameter (2097152    # 2 MBytes.) for ORB configuration option giopMaxMsgSize, reason: Invalid value, expect n >= 8192 or n == 0


RuntimeError: unidentifiable C++ exception

In [ ]:
%%bash
cd ../../ska-mid-dish-spfrx-talondx-console/
make oci-build-all

In [ ]:

import os
#Generate Config
os.system("docker run --rm --network host --volume " + SPFRX_MOUNT_DIR + ":/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:" + SPFRX_RELEASE + " ./spfrx_deployer.py --generate-spfrx-config")
#Download Artifacts
os.system("docker run --rm --env RAW_USER_ACCOUNT=" + RAW_USER_ACCOUNT + " --env RAW_USER_PASS=" + RAW_USER_PASS + " --volume " + SPFRX_MOUNT_DIR + ":/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:" + SPFRX_RELEASE + " ./spfrx_deployer.py --download-artifacts")
#Config DB
os.system("docker run --rm --network host --env TANGO_HOST=" + TANGO_HOST + " --volume " + SPFRX_MOUNT_DIR + ":/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:" + SPFRX_RELEASE + " ./spfrx_deployer.py --config-db")
print("Done!")

In [10]:
!../configs/spfrx/spfrx-deploy-artifacts.sh $SPFRX_ADDRESS $SPFRX_LOCAL_DIR $SPFRX_BIN $SPFRX_SCRIPTS_DIR

+ spfrx_ip=192.168.8.200
+ spfrx_local_dir=../configs/spfrx/mnt/spfrx-config/
+ spfrx_bin=/usr/local/bin
+ spfrx_scripts_dir=../configs/spfrx/
+ '[' 4 -lt 3 ']'
+ scp -o StrictHostKeyChecking=no ../configs/spfrx/mnt/spfrx-config//ska-mid-spfrx-controller-ds/bin/ska-mid-spfrx-controller-ds root@192.168.8.200:/usr/local/bin/.
../configs/spfrx/mnt/spfrx-config//ska-mid-spfrx-controller-ds/bin/ska-mid-spfrx-controller-ds: No such file or directory


In [18]:
!../configs/spfrx/program-bitstream-remote.sh $SPFRX_LOCAL_DIR $SPFRX_ADDRESS

+ spfrx_local_dir=../configs/spfrx/mnt/spfrx-config
+ board=192.168.8.200
+ path=/sys/kernel/config/device-tree/overlays
+ name=core
+ '[' 2 -lt 2 ']'
+ bs_core=talon_dx-spfrx_base-spfrx_core-hps_first.core.rbf
+ dtb=talon_dx-spfrx_base-spfrx_core.dtb
+ scp -o StrictHostKeyChecking=no ../configs/spfrx/mnt/spfrx-config/fpga-talon/bin/talon_dx-spfrx_base-spfrx_core-hps_first.core.rbf ../configs/spfrx/mnt/spfrx-config/fpga-talon/bin/talon_dx-spfrx_base-spfrx_core.dtb root@192.168.8.200:/lib/firmware
../configs/spfrx/mnt/spfrx-config/fpga-talon/bin/talon_dx-spfrx_base-spfrx_core-hps_first.core.rbf: No such file or directory
../configs/spfrx/mnt/spfrx-config/fpga-talon/bin/talon_dx-spfrx_base-spfrx_core.dtb: No such file or directory


In [10]:
# print("Checking SPFRx status\n If either of these pings fail, please see a system administrator")

# # PDU (for powercycling the SPFRx)
# #ssh apc@192.168.1.253
# #pswrd: apc
# #olStatus all
# #PSI_RXPU is the one we care about
# #OFF:
# #olOff 5
# #ON:
# #olOn 5
# #Pinging the clock
!ping 192.168.74.2 -c 1
#Pinging the SPF Surrogate
!ping 192.168.74.8 -c 1
#tango_host: 192.168.128.1
# #Stopping the RXPU incase it was already running
!../configs/spfrx/spfrx-remote-stop.sh 192.168.8.200 /usr/local/bin

# print("Configuring SPFRx...")
!../../ska-mid-dish-spfrx-talondx-console/make oci-build-all
#Generate SPFRx Config
!docker run --rm --network host --volume /$SPFRX_LOCAL_DIR:/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:$SPFRX_RELEASE ./spfrx_deployer.py --generate-spfrx-config
#Download Artifacts
!mkdir -p /$SPFRX_LOCAL_DIR
!docker run --rm --env RAW_USER_ACCOUNT=$RAW_USER_ACCOUNT --env RAW_USER_PASS=$RAW_USER_PASS --volume /$SPFRX_LOCAL_DIR:/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:$SPFRX_RELEASE ./spfrx_deployer.py --download-artifacts

#Program the bitstream
!../spfrx/program-bitstream-remote.sh $SPFRX_LOCAL_DIR $SPFRX_ADDRESS
#Config DB

!docker run --rm --network host --env TANGO_HOST=$TANGO_HOST --volume /$SPFRX_LOCAL_DIR:/app/images/ska-mid-dish-spfrx-talondx-console-deploy/artifacts:rw artefact.skao.int/ska-mid-dish-spfrx-talondx-console-deploy:$SPFRX_RELEASE ./spfrx_deployer.py --config-db
print("Done!")

print("Starting SPFRx...")
!../configs/spfrx/spfrx-remote-start.sh $TANGO_HOST $SPFRX_ADDRESS $SPFRX_BIN $SPFRX_TANGO_INSTANCE $SPFRX_DEFAULT_LOGGING_LEVEL

PING 192.168.74.2 (192.168.74.2) 56(84) bytes of data.
64 bytes from 192.168.74.2: icmp_seq=1 ttl=100 time=2.00 ms

--- 192.168.74.2 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 1.995/1.995/1.995/0.000 ms
PING 192.168.74.8 (192.168.74.8) 56(84) bytes of data.
64 bytes from 192.168.74.8: icmp_seq=1 ttl=100 time=0.145 ms

--- 192.168.74.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.145/0.145/0.145/0.000 ms
+ spfrx_ip=192.168.8.200
+ spfrx_bin=/usr/local/bin
+ '[' 2 -lt 2 ']'
+ echo 'BRINGDOWN RXPU ON 192.168.8.200'
BRINGDOWN RXPU ON 192.168.8.200
+ ssh -o StrictHostKeyChecking=no root@192.168.8.200 /usr/local/bin/spfrx-stop
Stopping SKA Mid SPFRx Controller Device Server pid=1382 ...
sh: 0: unknown operand
Unable to find SKA Mid SPFRx Low Level Device Server process, skipping ...
Stopping SKA Mid Talon-DX FPGA Temperature Monitor Device Server pid=1376 ...
sh: 0: unknown

### Load the Dish Vcc Config / Init Sys Params

In [ ]:
with open(f"{CONFIG_PATH}/{DISH_CONFIG_FILE}") as f:
    dish_config_json = f.read()

tmc_central_node.LoadDishCfg(dish_config_json)

print(f"TMC CSP Master Dish Vcc Config: \n{tmc_csp_master.dishVccConfig}")
print(f"\nTMC CSP Master Source Dish Vcc Config: \n{tmc_csp_master.sourceDishVccConfig}")

with open(f"{CONFIG_PATH}/{INIT_SYS_PARAM_FILE}") as f:
    cbf_init_sys_params_json = f.read()

cbf_controller.InitSysParam(cbf_init_sys_params_json)

print(f"\nCBF Init Sys Params: \n{cbf_subarray.sysParam}")

### Configure CBF Controller with the SLIM config for 4-receptors

In [ ]:
print("Copying the appropriate SLIM config for 4-receptors into the CBF Controller pod")
!kubectl exec -ti -n {NS} ds-cbfcontroller-controller-0 -- /bin/bash -c "mkdir -p mnt/slim"
!kubectl cp {CONFIG_PATH}/{SLIM_CONFIG_FILE} {NS}/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml

### Turn the Telescope On

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

print("Wait... ")

# Monitor the kubectl logs for the CBF Controller
!kubectl logs -n {NS} ds-cbfcontroller-controller-0 -f

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")

### Verify the Boards are On

SSH into each of the `TARGET_TALONS` to ensure they started correctly and run a "ps -ef" command on the board to ensure that the lower level device server executables have been copied over and are running.

### Generate the BITE Data using Engineering Console Bite

In [ ]:
print("Generating the BITE data")
!kubectl exec -ti -n {NS} ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards={TARGET_TALONS} --bite_mac_address={BITE_MAC_ADDRESS}

print("DONE")

### Monitor Visibilities Pod

These steps should be done in a separate terminal to monitor the packets when replaying the BITE data command a bit below.

In [ ]:
# kubectl apply -f viz_pod.yaml
# kubectl exec -ti sdn-dynamic-vis-1  -- bash
# apt update && apt install -y iproute2 tcpdump
# ip a 

# Update the IP in the Configure Scan to be the IP of the inet interface of the visibilities pod 

# tcpdump -i net1

### Assign Resources

In [ ]:
print("Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned")

with open(f"{CONFIG_PATH}/{ASSIGN_RESOURCES_FILE}") as f:
    assign_resources_json = f.read()

tmc_csp_subarray.AssignResources(assign_resources_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Configure Scan

In [ ]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(f"{CONFIG_PATH}/{CONFIGURE_SCAN_FILE}") as f:
    configure_scan_json = f.read()

tmc_csp_subarray.Configure(configure_scan_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

### Replay the BITE Data using Engineering Console Bite

In [ ]:
print("Replay the BITE Data")

!kubectl exec -ti -n {NS} ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards={TARGET_TALONS}

print("DONE")

### Scan

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(f"{CONFIG_PATH}/{SCAN_FILE}") as f:
    scan_json = f.read()

tmc_csp_subarray.Scan(scan_json)

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

### Clean Up


In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_csp_subarray.EndScan()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_csp_subarray.End()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")

In [ ]:
print("Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty")

tmc_csp_subarray.ReleaseAllResources()

print(f"CBF Subarray Observation State: {cbf_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")